In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import firebase_admin
from firebase_admin import credentials, db
import time

# Step 1: Load and preprocess the Excel file
file_path = r'C:\Users\Fady Kaisar\Desktop\New folder (2)\capstone\Capstone G12\ML\Recipe_and_Properties_of_Leathers\Data - Copy - Copy.xlsx'  # Replace with your file path
data = pd.read_excel(file_path)

# Filter and clean the data
filtered_data = data[(data['Temperature (°C)'] >= 0) & (data['Temperature (°C)'] <= 100)]
filtered_data = filtered_data[filtered_data['Sodium_sulfide'] >= 0]

# Define features and target
X = filtered_data[['Temperature (°C)', 'Soaking_pH']]
y = filtered_data['Sodium_sulfide']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Train the machine learning model
# Add polynomial features
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Train the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_poly, y_train)

# Evaluate the model
y_pred = model.predict(X_test_poly)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Model Evaluation: MSE = {mse}, R² = {r2}")

# Step 3: Integrate with Firebase
# Initialize Firebase Admin SDK
cred = credentials.Certificate("credentials.json")  # Replace with your Firebase Admin SDK JSON path
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://ml-soaking-app-default-rtdb.firebaseio.com/'  # Replace with your Firebase Database URL
})
firebase_ref = db.reference('/')

print("Model trained and Firebase initialized. Ready for real-time predictions.")

# Step 4: Real-time predictions from Firebase
while True:
    try:
        # Fetch the latest data from Firebase
        temperature = firebase_ref.child("Temperature").get()
        ph_value = firebase_ref.child("pH").get()

        if temperature is None or ph_value is None:
            print("Waiting for valid data in Firebase...")
            time.sleep(5)
            continue

        print(f"Fetched Data - Temperature: {temperature}, pH: {ph_value}")

        # Prepare data for prediction
        input_data = np.array([[temperature, ph_value]])
        input_data_poly = poly.transform(input_data)

        # Make prediction
        sodium_sulfide_pred = model.predict(input_data_poly)[0]
        print(f"Predicted Sodium Sulfide: {sodium_sulfide_pred}")

        # Upload prediction back to Firebase
        firebase_ref.child("Prediction").set({
            "Sodium_sulfide": sodium_sulfide_pred
        })
        print("Prediction uploaded to Firebase.")

    except Exception as e:
        print(f"Error: {e}")

    # Wait for 10 seconds before fetching new data
    time.sleep(10)


Model Evaluation: MSE = 0.09684818796670734, R² = 0.7700217036699212


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.